In [1]:
using DataFrames, StatsBase 

Useful Functions

In [2]:
#model is model_function
function out_sample_error(w,model,test_set)
    test_size = size(test_set,1)
    model_X = model(test_set)[2]
    model_output = round(model_X*w)
    model_error = 0
    for i in 1:test_size
        if test_set[:Survived][i] != model_output[i]
            model_error = model_error + 1
        end
    end
    return model_error / test_size
end

#model is a model function
function bootstrap_estimate(model,training_data
    ;resample_num = 1000)
    w_bs = 0
    td_size = size(training_data,1)
    for i in 1:resample_num
        train_sample = training_data[sample(1:td_size,round(Int,.5*td_size)), :] 
        w_bs = w_bs + model(train_sample)[1]
    end
    w_bs = w_bs / resample_num
    return w_bs
end 

#model is a model function
function cross_validate(model, training_data;
    k = 5)
    error_cv = 0
    td_size = size(training_data,1)
    partition = round(Int,td_size / k)
    for i in 1:k
        #creates training and test sets for this iteration
        s = (i-1)*partition + 1 
    if i == k 
            f = td_size
    else
        f = i*partition
    end
        round_test = training_data[s:f,:]
        round_train_1 = training_data[1:s-1,:]
        round_train_2 = training_data[f+1:td_size,:]
        round_train = [round_train_1;round_train_2]
        round_w = model(round_train)[1]
        round_error = out_sample_error(round_w,model,round_test)
        error_cv = error_cv + round_error
    end
    error_cv = error_cv / k
    return error_cv
end

#helper function for split_data
function build_subset(index,data)
    subset = similar(data,0)
    for i in 1:length(index)
        new_row = DataArray(data[data[:x] .== index[i],:])
        push!(subset,new_row)
    end
    return subset
end 

#splits data based on the discharge identifier column :x (integer)
function split_data(data_set;split_ratio = .8)
    n = size(data_set,1)
    split = Int(round(split_ratio*n))
    rand_index = shuffle(data_set[:x])
    train_index = rand_index[1:split]
    test_index = rand_index[split + 1:n]

    train = build_subset(train_index,data_set)
    test = build_subset(test_index,data_set)
    return(train,test)
end 


split_data (generic function with 1 method)

Analysis

In [3]:
heart_attack = readtable("heart_attack.csv")

,x,Health_Service_Area,Hospital_County,Operating_Certificate_Number,Facility_Id,Facility_Name,Age_Group,Zip_Code_3_digits,Gender,Race,Ethnicity,Length_of_Stay,Admit_Day_of_Week,Type_of_Admission,Patient_Disposition,Discharge_Year,Discharge_Day_of_Week,CCS_Diagnosis_Code,CCS_Diagnosis_Description,CCS_Procedure_Code,CCS_Procedure_Description,APR_DRG_Code,APR_DRG_Description,APR_MDC_Code,APR_MDC_Description,APR_Severity_of_Illness_Code,APR_Severity_of_Illness_Description,APR_Risk_of_Mortality,APR_Medical_Surgical_Description,Payment_Typology_1,Payment_Typology_2,Payment_Typology_3,Attending_Provider_License_Number,Operating_Provider_License_Number,Other_Provider_License_Number,Birth_Weight,Abortion_Edit_Indicator,Emergency_Department_Indicator,Total_Charges,Total_Costs,Survived,Male,White,Black,Other_Race,private_insurance,medicare,medicaid,old50_69,old70
1,3577,Western NY,Cattaraugus,401001,66,Olean General Hospital,70 or Older,147,M,White,Not Span/Hispanic,9,TUE,Emergency,Short-term Hospital,2012,THU,107,CARDIAC ARREST & VF,227,"OT DX PRC (INTERVW,EVAL",196,CARDIAC ARREST,5,Diseases and Disorders of the Circulatory System,2,Moderate,Moderate,Medical,Medicare,Medicaid,,264217,149079,NA,0,N,Y,$18461.03,$5148.18,1,1,1,0,0,0,1,0,0,1
2,5411,Western NY,Cattaraugus,401001,66,Olean General Hospital,18 to 29,147,M,White,Not Span/Hispanic,2,TUE,Emergency,Expired,2012,THU,107,CARDIAC ARREST & VF,227,"OT DX PRC (INTERVW,EVAL",196,CARDIAC ARREST,5,Diseases and Disorders of the Circulatory System,3,Major,Moderate,Medical,Self-Pay,,,141658,3053,NA,0,N,Y,$16729.38,$7543.35,0,1,1,0,0,0,0,0,0,0
3,8169,Western NY,Chautauqua,601000,98,Brooks Memorial Hospital,70 or Older,147,M,White,Not Span/Hispanic,1,SAT,Emergency,Expired,2012,SUN,107,CARDIAC ARREST & VF,229,NONOP RMVL FOREIGN BODY,196,CARDIAC ARREST,5,Diseases and Disorders of the Circulatory System,3,Major,Extreme,Medical,Medicare,,,239319,217335,217335,0,N,Y,$7145.49,$6382.73,0,1,1,0,0,0,1,0,0,1
4,9390,Western NY,Chautauqua,601000,98,Brooks Memorial Hospital,70 or Older,141,F,White,Not Span/Hispanic,2,MON,Urgent,Expired,2012,WED,107,CARDIAC ARREST & VF,216,RESP INTUB/MECH VENTIL,196,CARDIAC ARREST,5,Diseases and Disorders of the Circulatory System,3,Major,Major,Medical,Blue Cross/Blue Shield,,,261189,240432,NA,0,N,N,$6343.38,$10022.09,0,0,1,0,0,0,0,0,0,1
5,16311,Western NY,Chautauqua,602001,103,Woman's Christian Association,50 to 69,147,M,White,Not Span/Hispanic,5,FRI,Urgent,Home or Self Care,2012,WED,107,CARDIAC ARREST & VF,225,CONV OF CARDIAC RHYTHM,196,CARDIAC ARREST,5,Diseases and Disorders of the Circulatory System,4,Extreme,Major,Medical,Medicare,Medicare,,138697,129581,NA,0,N,Y,$18324.66,$9180.61,1,1,1,0,0,0,1,0,1,0
6,17014,Western NY,Chautauqua,602001,103,Woman's Christian Association,70 or Older,147,M,White,Not Span/Hispanic,2,WED,Urgent,Home or Self Care,2012,FRI,107,CARDIAC ARREST & VF,227,"OT DX PRC (INTERVW,EVAL",196,CARDIAC ARREST,5,Diseases and Disorders of the Circulatory System,2,Moderate,Moderate,Medical,Medicare,Medicare,,142873,172738,NA,0,N,Y,$6871.94,$2752.50,1,1,1,0,0,0,1,0,0,1
7,17754,Western NY,Chautauqua,602001,103,Woman's Christian Association,70 or Older,147,M,White,Not Span/Hispanic,4,FRI,Urgent,Skilled Nursing Home,2012,TUE,107,CARDIAC ARREST & VF,177,COMP AXIAL TOMOGR (CT),196,CARDIAC ARREST,5,Diseases and Disorders of the Circulatory System,2,Moderate,Moderate,Medical,Medicare,Medicare,,3876,130444,NA,0,N,Y,$7905.27,$3959.88,1,1,1,0,0,0,1,0,0,1
8,19217,Western NY,Chautauqua,427000,114,TLC Health Network Lake Shore Hospital,70 or Older,141,M,White,Not Span/Hispanic,1,TUE,Emergency,Expired,2012,TUE,107,CARDIAC ARREST & VF,216,RESP INTUB/MECH VENTIL,196,CARDIAC ARREST,5,Diseases and Disorders of the Circulatory System,3,Major,Major,Medical,Medicare,,,200012,118047,NA,0,N,Y,$3810.29,$1638.08,0,1,1,0,0,0,1,0,0,1
9,21080,Western NY,Erie,1401014,207,Buffalo General Hospital,50 to 69,141,M,White,Not Span/Hispanic,3,TUE,Urgent,Home or Self Care,2012,FRI,107,CARDIAC ARRE

Data Cleaning

In [4]:
heart_attack[heart_attack[:Length_of_Stay] .== "120 +",12] = "120"
heart_attack[:Length_of_Stay] = float(heart_attack[:Length_of_Stay])

1072-element Array{Float64,1}:
  9.0
  2.0
  1.0
  2.0
  5.0
  2.0
  4.0
  1.0
  3.0
  3.0
 47.0
  6.0
  4.0
  ⋮  
  6.0
 21.0
  5.0
  1.0
  1.0
  1.0
  2.0
  1.0
  2.0
 20.0
 21.0
  1.0

Split into train and test

In [5]:
#LI_data = heart_attack[heart_attack[:Health_Service_Area] .== "Long Island",:]
NYC_data = heart_attack[heart_attack[:Health_Service_Area] .== "New York City",:]
train = split_data(NYC_data)[1]
test = split_data(NYC_data)[2]

,x,Health_Service_Area,Hospital_County,Operating_Certificate_Number,Facility_Id,Facility_Name,Age_Group,Zip_Code_3_digits,Gender,Race,Ethnicity,Length_of_Stay,Admit_Day_of_Week,Type_of_Admission,Patient_Disposition,Discharge_Year,Discharge_Day_of_Week,CCS_Diagnosis_Code,CCS_Diagnosis_Description,CCS_Procedure_Code,CCS_Procedure_Description,APR_DRG_Code,APR_DRG_Description,APR_MDC_Code,APR_MDC_Description,APR_Severity_of_Illness_Code,APR_Severity_of_Illness_Description,APR_Risk_of_Mortality,APR_Medical_Surgical_Description,Payment_Typology_1,Payment_Typology_2,Payment_Typology_3,Attending_Provider_License_Number,Operating_Provider_License_Number,Other_Provider_License_Number,Birth_Weight,Abortion_Edit_Indicator,Emergency_Department_Indicator,Total_Charges,Total_Costs,Survived,Male,White,Black,Other_Race,private_insurance,medicare,medicaid,old50_69,old70
1,1437432,New York City,Kings,7001037,1320,University Hospital of Brooklyn,50 to 69,112,M,Black/African American,Not Span/Hispanic,7.0,TUE,Emergency,Home or Self Care,2012,TUE,107,CARDIAC ARREST & VF,231,OTHER THERAPEUTIC PRCS,196,CARDIAC ARREST,5,Diseases and Disorders of the Circulatory System,3,Major,Moderate,Medical,Medicare,Medicare,Self-Pay,219902,258334,NA,0,N,Y,$18907.00,$13284.58,1,1,0,1,0,0,1,0,1,0
2,1749073,New York City,Manhattan,7002054,1458,New York Presbyterian Hospital - New York Weill Cornell Center,18 to 29,115,M,Other Race,Not Span/Hispanic,21.0,TUE,Emergency,Home w/ Home Health Services,2012,TUE,107,CARDIAC ARREST & VF,160,OT THER PRCS ON MUSCLES,180,OTHER CIRCULATORY SYSTEM PROCEDURES,5,Diseases and Disorders of the Circulatory System,4,Extreme,Extreme,Surgical,Medicaid,Medicaid,Self-Pay,205827,210584,NA,0,N,Y,$170006.00,$53078.35,1,1,0,0,1,0,0,1,0,0
3,1376960,New York City,Kings,7001021,1306,New York Methodist Hospital,70 or Older,114,M,Other Race,Not Span/Hispanic,50.0,FRI,Emergency,Expired,2012,SAT,107,CARDIAC ARREST & VF,45,PERC TRANSLUM COR ANGIO,4,TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE PROCEDURE OR ECMO,5,Diseases and Disorders of the Circulatory System,4,Extreme,Extreme,Surgical,Medicare,Medicare,Self-Pay,175965,251309,NA,0,N,N,$300829.68,$149836.17,0,1,0,0,1,0,1,0,0,1
4,2089898,New York City,Queens,7002024,1639,Mount Sinai Hospital - Mount Sinai Hospital of Queens,50 to 69,111,F,Other Race,Not Span/Hispanic,2.0,FRI,Emergency,Expired,2012,SUN,107,CARDIAC ARREST & VF,216,RESP INTUB/MECH VENTIL,196,CARDIAC ARREST,5,Diseases and Disorders of the Circulatory System,4,Extreme,Extreme,Medical,Medicaid,,,245517,245517,NA,0,N,Y,$15947.00,$6434.32,0,0,0,0,1,0,0,1,1,0
5,978950,New York City,Bronx,7000002,1165,Jacobi Medical Center,50 to 69,100,M,Other Race,Not Span/Hispanic,9.0,WED,Emergency,Expired,2012,FRI,107,CARDIAC ARREST & VF,216,RESP INTUB/MECH VENTIL,196,CARDIAC ARREST,5,Diseases and Disorders of the Circulatory System,4,Extreme,Extreme,Medical,Medicare,Medicare,Medicaid,259789,259789,NA,0,N,Y,$28314.22,$22264.55,0,1,0,0,1,0,1,0,1,0
6,980106,New York City,Bronx,7000002,1165,Jacobi Medical Center,70 or Older,104,M,Other Race,Not Span/Hispanic,21.0,WED,Emergency,Another Type Not Listed,2012,WED,107,CARDIAC ARREST & VF,216,RESP INTUB/MECH VENTIL,196,CARDIAC ARREST,5,Diseases and Disorders of the Circulatory System,4,Extreme,Extreme,Medical,Medicare,Medicare,Medicaid,3872,3872,NA,0,N,Y,$90722.92,$71338.88,1,1,0,0,1,0,1,0,0,1
7,1024361,New York City,Bronx,7000006,1169,Montefiore Medical Center - Henry & Lucy Moses Div,50 to 69,104,M,Other Race,Spanish/Hispanic,3.0,SAT,Emergency,Home or Self Care,2012,TUE,107,CARDIAC ARREST & VF,48,CARDIAC PACEMAKER/DEFIB,161,CARDIAC DEFIBRILLATOR & HEART ASSIST IMPLANT,5,Diseases and Disorders of the Circulatory System,1,Minor,Minor,Surgical,Private Health Insurance,Self-Pay,,250183,110785,NA,0,N,Y,$87057.65,$24756.44,1,1,0,0,1,1,0,0,1,0
8,1916754,New York City,Queens,7003000,1626,Elmhurst Hospital Center,70 or Older,113,M,White,Not Span/Hispanic,11.0,SAT,Emergency,Short-term Hospital,2012,WED,107,CARDIAC ARREST & VF,34,TRA

Fit models on training data

In [6]:
function model_one(training_data)
    td_size = size(training_data,1)
    X = [training_data[:APR_Severity_of_Illness_Code].data training_data[:Length_of_Stay] ones(td_size)]
    y = training_data[:Survived].data
    w = X\y
    return (w,X)
end 

function model_two(training_data)
    td_size = size(training_data,1)
    X = [training_data[:APR_Severity_of_Illness_Code].data training_data[:Length_of_Stay] training_data[:Male] ones(td_size)]
    y = training_data[:Survived].data
    w = X\y
    return (w,X)
end 

function model_three(training_data)
    td_size = size(training_data,1)
    X = [training_data[:APR_Severity_of_Illness_Code].data training_data[:Length_of_Stay] training_data[:Male] training_data[:old70] ones(td_size)]
    y = training_data[:Survived].data
    w = X\y
    return (w,X)
end 

w1 = model_one(train)[1]
w2 = model_two(train)[1]
w3 = model_three(train)[1]



5-element Array{Float64,1}:
 -0.263324 
  0.0212358
  0.171302 
 -0.177788 
  1.07332  

cross validation of models

In [7]:
@show cross_validate(model_one, train)
@show cross_validate(model_two, train) 
@show cross_validate(model_three, train) 

cross_validate(model_one,train) = 0.22836463223787168
cross_validate(model_two,train) = 0.1949530516431925
cross_validate(model_three,train) = 0.19769170579029732


0.19769170579029732

Bootstrap estimate for models

In [8]:
w1_bs = bootstrap_estimate(model_one,train)
w2_bs = bootstrap_estimate(model_two,train)
w3_bs = bootstrap_estimate(model_three,train)

5-element Array{Float64,1}:
 -0.266392 
  0.0224102
  0.169598 
 -0.175578 
  1.07726  

Check model performance on validation set

In [9]:
@show out_sample_error(w1,model_one,test)
@show out_sample_error(w1_bs,model_one,test)
@show out_sample_error(w2,model_two,test)
@show out_sample_error(w2_bs,model_two,test)
@show out_sample_error(w3,model_three,test)
@show out_sample_error(w3_bs,model_three,test)

out_sample_error(w1,model_one,test) = 0.25555555555555554
out_sample_error(w1_bs,model_one,test) = 0.2111111111111111
out_sample_error(w2,model_two,test) = 0.13333333333333333
out_sample_error(w2_bs,model_two,test) = 0.13333333333333333
out_sample_error(w3,model_three,test) = 0.14444444444444443
out_sample_error(w3_bs,model_three,test) = 0.15555555555555556


0.15555555555555556

Things to add: try adding regulizers to some models/generalize model making process
Ways to measure model error in terms of bias and variance 